# Workers displacement heatmap 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np
import json
import ast
import pandas as pd
import os
from scipy import ndimage
from skimage import io
from skimage import transform

## Path to analytics data

In [ ]:
images_analytics = PATH_TO_ANALYTICS_IMG
labels_analytics_poly = PATH_TO_ANALYTICS_JSON_POLY #shape: bitmap + origin point
labels_analytics_people = PATH_TO_ANALYTICS_JSON_PEOPLE #shape: bounding box left upper and right lower point

In [ ]:
sorted_images = [images_analytics+image for image in sorted(os.listdir(images_analytics))]
sorted_labels_poly = [labels_analytics_poly+label for label in sorted(os.listdir(labels_analytics_poly))]
sorted_labels_people = [labels_analytics_people+label for label in sorted(os.listdir(labels_analytics_people))]

## Heatmap

In [ ]:
def add(image, heat_map, alpha=0.6, display=False, save=None, cmap='coolwarm', axis='on', verbose=False):

    height = image.shape[0]
    width = image.shape[1]

    # resize heat map
    heat_map_resized = transform.resize(heat_map, (height, width))

    # normalize heat map
    max_value = np.max(heat_map_resized)
    min_value = np.min(heat_map_resized)
    normalized_heat_map = (heat_map_resized - min_value) / (max_value - min_value)

    # display
    plt.imshow(image)
    plt.imshow(255 * normalized_heat_map, alpha=alpha, cmap=cmap)
    plt.axis(axis)

    if display:
        plt.show()

    if save is not None:
        if verbose:
            print('save image: ' + save)
        plt.savefig(save, bbox_inches='tight', pad_inches=0)

In [ ]:
heat = [[0 for i in range(1280)] for i in range(1080)]
for text_file in sorted_labels_people:
    f = open(text_file)
    text = f.read()
    dic = ast.literal_eval(text)
    objects = dic['objects']
    
    for obj in objects:
        if obj['classTitle'] == 'People_model':
            #mid_x =  (obj['points']['exterior'][0][0] + obj['points']['exterior'][1][0])/2
            #mid_y =  (obj['points']['exterior'][0][1] + obj['points']['exterior'][1][1])/2
            upper_right =  obj['points']['exterior'][1]
            lower_left =  obj['points']['exterior'][0]
            
            for i in range (lower_left[0], upper_right[0]+1):
                for j in range (lower_left[1], upper_right[1]+1):
                    try:
                        heat[j][i]+=1
                    except:
                        print(i,j)
                        
# read image on which to put the heatmap
image = io.imread(sorted_images[15])

# create heat map
add(image, np.array(heat), alpha=0.7, save='heat_map.png')